# Llama 2 Medical Finetuned

## Step 1: Installing and importing the libraries

In [ ]:
!pip uninstall accelerate peft bitsandbytes transformers trl -y
!pip install accelerate peft==0.13.2 bitsandbytes transformers trl==0.12.0

Found existing installation: accelerate 1.9.0
Uninstalling accelerate-1.9.0:
  Successfully uninstalled accelerate-1.9.0
Found existing installation: peft 0.13.2
Uninstalling peft-0.13.2:
  Successfully uninstalled peft-0.13.2
Found existing installation: bitsandbytes 0.46.1
Uninstalling bitsandbytes-0.46.1:
  Successfully uninstalled bitsandbytes-0.46.1
Found existing installation: transformers 4.55.0
Uninstalling transformers-4.55.0:
  Successfully uninstalled transformers-4.55.0
Found existing installation: trl 0.12.0
Uninstalling trl-0.12.0:
  Successfully uninstalled trl-0.12.0
  Using cached accelerate-1.9.0-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.13.2-py3-none-any.whl.metadata (13 kB)
  Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
  Using cached transformers-4.55.0-py3-none-any.whl.metadata (39 kB)
  Using cached trl-0.12.0-py3-none-any.whl.metadata (10 kB)
Using cached peft-0.13.2-py3-none-any.whl (320 kB)
Using cached trl

In [ ]:
!pip install huggingface_hub

In [1]:
import torch
from trl import SFTTrainer
from peft import LoraConfig, PeftModel
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

### Environment configuration for guaranteed GPU runtime.

In [2]:
torch.cuda.empty_cache()
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


## Step 2: Loading the model

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

llama_model = AutoModelForCausalLM.from_pretrained(
    "aboonaji/llama2finetune-v2",
    quantization_config=bnb_config,
    torch_dtype=torch.float16
)

llama_model.config.use_cache = False
llama_model.config.pretraining_tp = 1

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Step 3: Loading the tokenizer

In [4]:
llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2", trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

## Step 3.5 (Optional): Load the Model Weights

In [5]:
llama_model = PeftModel.from_pretrained(llama_model, "./results/checkpoint-100")

## Step 4: Setting the training arguments

In [ ]:
training_arguments = TrainingArguments(output_dir = "./results", per_device_train_batch_size = 1, gradient_accumulation_steps = 4, max_steps = 100)

## Step 5: Creating the Supervised Fine-Tuning trainer

In [ ]:
llama_sft_trainer = SFTTrainer(model = llama_model,
                               args = training_arguments,
                               train_dataset = load_dataset(path = "aboonaji/wiki_medical_terms_llam2_format", split = "train"),
                               tokenizer = llama_tokenizer,
                               peft_config = LoraConfig(task_type = "CAUSAL_LM", r = 16, lora_alpha = 16, lora_dropout = 0.1),
                               dataset_text_field = "text")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


## Step 6: Training the model

In [ ]:
llama_sft_trainer.train()

wandb: Currently logged in as: malizdmr011 (malizdmr011-bogazici) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=100, training_loss=1.6638002014160156, metrics={'train_runtime': 2336.3223, 'train_samples_per_second': 0.171, 'train_steps_per_second': 0.043, 'total_flos': 1.090662998716416e+16, 'train_loss': 1.6638002014160156, 'epoch': 0.058300539279988337})

## Step 7: Chatting with the model

In [6]:
user_prompt = "Please tell me about Tinea Pedis"
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 500)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])

Device set to use cuda:0


<s>[INST] Please tell me about Tinea Pedis [/INST]  Tinea pedis, also known as athlete's foot, is a common fungal infection of the skin that affects the feet. hopefully, this information will help you understand more about this condition.

Causes:
Tinea pedis is caused by a fungus called dermatophyte, which thrives in warm, moist environments. The fungus feeds on keratin, a protein found in skin, hair, and nails. It can also be spread through direct contact with an infected person or contaminated objects.

Symptoms:
The symptoms of tinea pedis can vary depending on the severity of the infection. Common symptoms include:

* Redness and itching on the skin
* Cracking and peeling skin
* Thickening and discoloration of the skin
* Blisters or sores on the skin
* Pain or discomfort in the feet
* A foul odor emanating from the feet
* White or yellowish patches on the skin
* Skin thickening and scaling

Risk factors:
Tinea pedis can affect anyone, but certain factors increase the risk of devel